# Data Exploratory Analysis
The following outlines the process I used to understand and analyze the dataset.

In [4]:
# The first step involves importing the libraries required for the process:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# The graphics style selected is:
plt.style.use('ggplot')

In [6]:
# The following allows us to view all the columns of the dataset, regardless of its size:
pd.set_option('display.max_columns', None)

In [8]:
# Then the dataset is loaded as:
corn = pd.read_csv("C://Users/jober/Data_Projects/corn-yield-prediction/Dataset/corn_data.csv", sep=";", )

## Step 1: Understanding the data
This step give us a general sense of the dataset: 

In [9]:
corn.shape

(422, 22)

In [10]:
corn.head()

,County,Farmer,Education,Gender,Age bracket,Household size,Crop,Acreage,Fertilizer amount,Laborers,Yield,Power source,Water source,Main credit source,Crop insurance,Farm records,Main advisory source,Extension provider,Advisory format,Advisory language,Latitude,Longitude
0,TAITA TAVETA,fmr_65,Certificate,Male,36-45,7,corn,2.00,50,2,300,Manual,Rain,Credit groups,No,Yes,Radio,Private Provider,Phone Calls,Vernacular,-3.46,38.35
1,TAITA TAVETA,fmr_77,Certificate,Male,36-45,7,corn,0.25,50,2,270,Manual,Rain,Credit groups,No,Yes,Radio,County Government,SMS text,Kiswahili,-3.31,38.40
2,TAITA TAVETA,fmr_89,Certificate,Male,36-45,7,corn,3.00,251,2,270,Manual,Rain,Credit groups,No,Yes,Radio,Private Provider,Phone Calls,Vernacular,-3.41,38.37
3,TAITA TAVETA,fmr_102,Certificate,Male,36-45,7,corn,1.50,300,3,200,Manual,Rain,Credit groups,No,Yes,Radio,County Government,SMS text,Kiswahili,-3.39,38.37
4,TAITA TAVETA,fmr_25,Certificate,Male,46-55,3,corn,NaN,50,2,180,Manual,Rain,Credit groups,No,Yes,Radio,Private Provider,Phone Calls,Vernacular,-3.39,38.33


In [11]:
corn.columns

Index(['County', 'Farmer', 'Education', 'Gender', 'Age bracket',
       'Household size', 'Crop', 'Acreage', 'Fertilizer amount', 'Laborers',
       'Yield', 'Power source', 'Water source', 'Main credit source',
       'Crop insurance', 'Farm records', 'Main advisory source',
       'Extension provider', 'Advisory format', 'Advisory language',
       'Latitude', 'Longitude'],
      dtype='object')

In [13]:
# Using the info() method, we can quickly identify the data type of each column and detect null values:"
corn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   County                422 non-null    object 
 1   Farmer                422 non-null    object 
 2   Education             396 non-null    object 
 3   Gender                422 non-null    object 
 4   Age bracket           422 non-null    object 
 5   Household size        422 non-null    int64  
 6   Crop                  422 non-null    object 
 7   Acreage               351 non-null    float64
 8   Fertilizer amount     422 non-null    int64  
 9   Laborers              422 non-null    int64  
 10  Yield                 422 non-null    int64  
 11  Power source          422 non-null    object 
 12  Water source          422 non-null    object 
 13  Main credit source    422 non-null    object 
 14  Crop insurance        422 non-null    object 
 15  Farm records          4

In [17]:
# The number of null values in the dataset is confirmed as:
corn.isna().sum()

County                   0
Farmer                   0
Education               26
Gender                   0
Age bracket              0
Household size           0
Crop                     0
Acreage                 71
Fertilizer amount        0
Laborers                 0
Yield                    0
Power source             0
Water source             0
Main credit source       0
Crop insurance           0
Farm records             0
Main advisory source     0
Extension provider       0
Advisory format          0
Advisory language        0
Latitude                 0
Longitude                0
dtype: int64

In [14]:
# The describe() function provides basic statistics for the numerical variables in the dataset:
corn.describe()

,Household size,Acreage,Fertilizer amount,Laborers,Yield,Latitude,Longitude
count,422.000000,351.000000,422.000000,422.000000,422.000000,422.000000,422.000000
mean,4.263033,0.828063,65.189573,2.843602,232.604265,-3.417583,38.404455
std,1.706355,0.584884,65.754796,0.974348,135.376952,0.079007,0.094324
min,1.000000,0.250000,20.000000,2.000000,50.000000,-3.640000,38.270000
25%,3.000000,0.500000,25.000000,2.000000,100.000000,-3.490000,38.340000
50%,4.000000,0.500000,50.000000,3.000000,180.000000,-3.410000,38.370000
75%,5.000000,1.000000,75.000000,3.000000,330.000000,-3.360000,38.400000
max,9.000000,4.000000,500.000000,8.000000,600.000000,-3.250000,38.630000


## Step 2: Data preparation
Now that I have a general understanding of the data, some cleaning is needed before proceeding with further analysis.

In [40]:
# The column 'Farmer' indicates a unique record for each of the 422 platantion leader's.
corn['Farmer'].value_counts()

Farmer
fmr_65     1
fmr_219    1
fmr_72     1
fmr_71     1
fmr_368    1
          ..
fmr_410    1
fmr_371    1
fmr_274    1
fmr_254    1
fmr_200    1
Name: count, Length: 422, dtype: int64

In [43]:
# In addition, the following columns are not useful for creating a predictive model because they return the same value for all rows, as shown below:
cols = ['County', 'Crop', 'Power source', 'Water source','Crop insurance']
for c in cols:
    print(corn[c].value_counts())

County
TAITA TAVETA    422
Name: count, dtype: int64
Crop
corn    422
Name: count, dtype: int64
Power source
Manual    422
Name: count, dtype: int64
Water source
Rain    422
Name: count, dtype: int64
Crop insurance
No    422
Name: count, dtype: int64


In [44]:
# Additionally, the columns 'Latitude' and 'Longitude' do not provide value due to their low variance within the analyzed county.

In [45]:
# Then, our subset selected for analysis is:
corn_subset = corn[['Education', 'Gender', 'Age bracket',
                    'Household size', 'Acreage', 'Fertilizer amount', 'Laborers',
                    'Yield', 'Main credit source', 'Farm records', 
                    'Main advisory source', 'Extension provider', 'Advisory format', 
                    'Advisory language']]
corn_subset.head()

,Education,Gender,Age bracket,Household size,Acreage,Fertilizer amount,Laborers,Yield,Main credit source,Farm records,Main advisory source,Extension provider,Advisory format,Advisory language
0,Certificate,Male,36-45,7,2.00,50,2,300,Credit groups,Yes,Radio,Private Provider,Phone Calls,Vernacular
1,Certificate,Male,36-45,7,0.25,50,2,270,Credit groups,Yes,Radio,County Government,SMS text,Kiswahili
2,Certificate,Male,36-45,7,3.00,251,2,270,Credit groups,Yes,Radio,Private Provider,Phone Calls,Vernacular
3,Certificate,Male,36-45,7,1.50,300,3,200,Credit groups,Yes,Radio,County Government,SMS text,Kiswahili
4,Certificate,Male,46-55,3,NaN,50,2,180,Credit groups,Yes,Radio,Private Provider,Phone Calls,Vernacular


In [52]:
# Column names for our more suitable dataframe are set as:
corn_subset.columns = [name.lower() for name in corn_subset.columns]
corn_subset.columns

Index(['education', 'gender', 'age bracket', 'household size', 'acreage',
       'fertilizer amount', 'laborers', 'yield', 'main credit source',
       'farm records', 'main advisory source', 'extension provider',
       'advisory format', 'advisory language'],
      dtype='object')

- dropping irrelevant columns and rows
- identifying dupplicated columns
- renaming columns
- feature creation
